[Advent of Code - Day 11](https://adventofcode.com/2019/day/11)

# Import input

In [1]:
import sys
from collections import defaultdict

sys.path.insert(0, "../")
from utils import aoc_input as inp

INPUT = inp.download_input(year="2019", day="11")[0]

In [2]:
INPUT[:5]

'3,8,1'

# Solulu

In [3]:
def solve(
    intcode: list,
    panel_color: int,
    pointer: int = 0,
    loc: set = (0, 0),
    direction: int = 0,
):
    colored = dict()
    directions = [(0, +1), (+1, 0), (0, -1), (-1, 0)]
    output_index = 0
    output_values = list()
    relative_base = 0

    while True:
        opcode, modes, total_values = decode(intcode[pointer])
        if opcode == 99:
            break

        elif opcode == 9:
            relative_base += parameter_mode(modes[0], pointer, 1, relative_base)

        elif opcode == 4:
            output_index = parameter_mode(
                modes[0], pointer, 1, relative_base, index=True
            )
            output_values.append(intcode[output_index])

            if len(output_values) == 2:
                color, right = output_values

                if not right:
                    direction = (direction - 1) % len(directions)
                else:
                    direction = (direction + 1) % len(directions)
                x, y = directions[direction]
                colored[loc] = color
                loc = (loc[0] + x, loc[1] + y)
                output_values = list()

        elif opcode == 3:
            input_value = colored.get(loc, panel_color)
            input_index = parameter_mode(
                modes[0], pointer, 1, relative_base, index=True
            )
            intcode[input_index] = input_value

        elif opcode in [1, 2, 7, 8]:
            nums = []
            for i, mode in enumerate(modes[:-1], start=1):
                num = parameter_mode(mode, pointer, i, relative_base)
                nums.append(num)
            if opcode == 1:
                value = nums[0] + nums[1]
            elif opcode == 2:
                value = nums[0] * nums[1]
            elif opcode == 7:
                value = nums[0] < nums[1]
            elif opcode == 8:
                value = nums[0] == nums[1]
            value_index = parameter_mode(
                modes[-1], pointer, 3, relative_base, index=True
            )
            intcode[value_index] = value

        if opcode == 5 and parameter_mode(modes[0], pointer, 1, relative_base) != 0:
            pointer = parameter_mode(modes[1], pointer, 2, relative_base)
        elif opcode == 6 and parameter_mode(modes[0], pointer, 1, relative_base) == 0:
            pointer = parameter_mode(modes[1], pointer, 2, relative_base)
        else:
            pointer += total_values

    return colored


def parameter_mode(mode, pointer, param, relative_base, index: bool = False) -> int:
    if mode == 0:
        num_index = intcode[pointer + param]
    elif mode == 1:
        num_index = pointer + param
    elif mode == 2:
        num_index = intcode[pointer + param] + relative_base

    return num_index if index else intcode[num_index]


def decode(instruction):
    opcode = instruction % 100
    modes = list()
    total_values = 1

    mode1 = (instruction // 100) % 10
    modes.append(mode1)
    if opcode not in (3, 4, 9):
        mode2 = instruction // 1000 % 10
        modes.append(mode2)
        if opcode not in (5, 6):
            mode3 = instruction // 10000
            modes.append(mode3)

    total_values += len(modes)

    return opcode, modes, total_values


def _reset_memory(inp=INPUT):
    intcode = list(map(int, INPUT.split(",")))
    d = defaultdict(int)
    for address, value in enumerate(intcode):
        d[address] = value
    return d

## Part 1

In [4]:
intcode = _reset_memory(INPUT)

colored = solve(intcode, panel_color=0)
len(colored)

1932

## Part 2

In [5]:
intcode = _reset_memory(INPUT)

colored = solve(intcode, panel_color=1)

In [6]:
minx, maxx = 0, 0
miny, maxy = 0, 0

for x, y in colored:
    minx = min(minx, x)
    maxx = max(maxx, x)
    miny = min(miny, y)
    maxy = max(maxy, y)

for y in range(maxy, miny - 1, -1):
    print()
    for x in range(minx, maxx + 1):
        if colored.get((x, y), 0) == 1:
            print("#", end=" ")
        else:
            print(" ", end=" ")


  # # # #     # #     #     #   #     #     # #         # #   # # # #   # # #         
  #         #     #   #     #   #   #     #     #         #   #         #     #       
  # # #     #         # # # #   # #       #               #   # # #     #     #       
  #         #   # #   #     #   #   #     #   # #         #   #         # # #         
  #         #     #   #     #   #   #     #     #   #     #   #         #   #         
  # # # #     # # #   #     #   #     #     # # #     # #     # # # #   #     #       